In [1]:
# Do some imports.
from astroquery.mast import Observations
import numpy

In [2]:
# Let's get every time series observation available at MAST within 0.2 degrees of M67.
search_result = Observations.query_criteria(objectname="M67", dataproduct_type=["timeseries"], radius="0.2 deg")
# The return result is a Table containing the metadata you see when doing a MAST Portal search: coordinates, timestamps,
# filters, target name, mission/project, etc.
print(search_result.columns)

<TableColumns names=('dataproduct_type','calib_level','obs_collection','obs_id','target_name','s_ra','s_dec','t_min','t_max','t_exptime','wavelength_region','filters','em_min','em_max','target_classification','obs_title','t_obs_release','dataRights','mtFlag','srcDen','instrument_name','proposal_pi','proposal_id','intentType','project','s_region','jpegURL','dataURL','obsid','objID','objID1','distance')>


In [3]:
# How many observations did we get?
print(len(search_result))

1436


In [4]:
# Did these all come from the K2 mission?  Answer: no!  High Level Science Products too!
print(set(search_result['project']))

{'hlsp_polar', 'hlsp_everest', 'K2', 'hlsp_k2sc', 'hlsp_k2sff'}


In [5]:
# Let's get a list of all the K2 IDs.  NOTE:  K2 mission-produced products always have "project" = "K2".
where_k2_mission = numpy.where(search_result['project'] == 'K2')[0]
print(search_result['target_name'][where_k2_mission][0:5])

 target_name 
-------------
ktwo200008665
ktwo200008692
ktwo211401366
ktwo211400464
ktwo211409348


In [6]:
# I just want the actual ID numbers, so make a numpy array stripping away the "ktwo" prefix.
k2_ids = numpy.asarray(
        [x.strip('ktwo') for x in search_result['target_name'][where_k2_mission]]
    )
print(k2_ids)

['200008665' '200008692' '211401366' '211400464' '211409348' '211400500'
 '200008654' '200008662' '200008672' '200008674' '200008685' '200008694'
 '200008695' '200008697' '211400150' '211401338' '211401610' '211408858'
 '211409011' '211409644' '211411922' '211412877' '211416749' '200008655'
 '200008664' '200008666' '200008675' '200008677' '200008684' '200008686'
 '200008689' '200008709' '211398530' '211399673' '211400811' '211408522'
 '211408750' '211409088' '211410931' '211412252' '211417765' '211417812'
 '211421683' '228682441' '200008656' '200008657' '200008658' '200008659'
 '200008667' '200008668' '200008671' '200008676' '200008679' '200008680'
 '200008681' '200008682' '200008688' '200008690' '200008691' '200008699'
 '200008700' '200008701' '200008704' '200008706' '200008711' '211398920'
 '211399364' '211399481' '211399819' '211400215' '211400364' '211400616'
 '211400671' '211401069' '211401367' '211401403' '211403555' '211408015'
 '211408116' '211408990' '211409560' '211411508' '2

In [7]:
# Now we will loop over each K2 ID and download the light curves if and only if there is a LC from the mission
# AND all four of the HLSP teams.
for k2id in k2_ids:
        where_k2id = numpy.where(search_result['target_name'] == 'ktwo' + k2id)[0]
        # Does this have a POLAR light curve?
        where_polar = numpy.where(search_result['target_name'] == 'polar' + k2id)[0]
        n_polar = len(where_polar)
        # Does this have a K2SFF light curve?
        where_k2sff = numpy.where(search_result['target_name'] == 'k2sff' + k2id)[0]
        n_k2sff = len(where_k2sff)
        # Does this have an EVEREST light curve?
        where_everest = numpy.where(search_result['target_name'] == 'everest' + k2id)[0]
        n_everest = len(where_everest)
        # Does this have a K2SC light curve?
        where_k2sc = numpy.where(search_result['target_name'] == 'k2sc' + k2id)[0]
        n_k2sc = len(where_k2sc)
        if n_polar == 1 and n_k2sff == 1 and n_everest == 1 and n_k2sc == 1:
                print("K2 ID " + k2id + " - All five light curves found!")
                Observations.download_products(search_result['obsid'][where_k2id])
                Observations.download_products(search_result['obsid'][where_polar])
                Observations.download_products(search_result['obsid'][where_k2sff])
                Observations.download_products(search_result['obsid'][where_everest])
                Observations.download_products(search_result['obsid'][where_k2sc])

# Note: here I am downloading one observation at a time using the "obsid", but download_products can also accept
# a list of obsid's, or even an entire Table of data products, so I could do the download step in a single line
# if I wanted.  Note also that if you add a "curl_flag = True" to the download_products() call, it will create a
# curl shell script you can execute later, if you don't want to actually download the files right away.
                

K2 ID 228682441 - All five light curves found!
K2 ID 228682410 - All five light curves found!
K2 ID 228682412 - All five light curves found!
